- Anmol More - 11915043
- Shlok Mittal - 11915071

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/model_data/trip_time_prediction.csv')
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df['hour'] = df['TIMESTAMP'].dt.hour
df['dow'] = df['TIMESTAMP'].dt.weekday
df.drop(columns = ['TIMESTAMP'], inplace=True)

X = df.loc[:, df.columns != 'Trip Time']
y = df['Trip Time']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(1200729, 7) (1200729,)
(300183, 7) (300183,)


In [3]:
%%time
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, 
                                max_depth=1, random_state=0, loss='ls').fit(X_train, y_train)
np.sqrt(mean_squared_error(y_test, est.predict(X_test)))
print ("Test Accuracy : ", est.score(X_test, y_test))

Test Accuracy :  0.5523607724419144
CPU times: user 7min 11s, sys: 10.9 s, total: 7min 22s
Wall time: 7min 34s


In [5]:
import pickle
# filename = '../model/triptime_prediction.sav'
# pickle.dump(est, open(filename, 'wb'))

#est = pickle.load(open(filename, 'rb'))
#result = est.score(X_test, y_test)
#print(result)

filename = 'model.pkl'
pickle.dump(est, open(filename, 'wb'))

In [6]:
def predict_trip_time(start_lat, start_long, end_lat, end_long, date = 0) :
    import datetime
    import pickle
    import numpy as np
        
    if(date == 0) :
        date = datetime.datetime.now()
        hour = date.hour
    else :
        date = datetime.datetime.strptime(date, "%Y-%m-%d")
        hour = 15
    dayofweek = date.weekday()

    filename = '../model/triptime_prediction.sav'
    gbr_model = pickle.load(open(filename, 'rb'))
    
    earth_radius = 6371
    lat1, lon1, lat2, lon2 = np.radians([start_lat, start_long, end_lat, end_long])
    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    distance = earth_radius * 2 * np.arcsin(np.sqrt(a))

    result = int(gbr_model.predict(np.array((start_lat, start_long, 
                                            end_lat, end_long, 
                                            distance, hour, dayofweek)).reshape(1, -1)))
    print(start_lat, start_long,end_lat, end_long,distance, hour, dayofweek)
    return result

start_lat = 41.158962
start_long = -8.634978
end_lat = 41.140584
end_long = -8.615817
print(predict_trip_time(start_lat, start_long, end_lat, end_long))
print(predict_trip_time(start_lat, start_long, end_lat, end_long, '2019-11-24'))


41.158962 -8.634978 41.140584 -8.615817 2.598062705571137 21 4
453
41.158962 -8.634978 41.140584 -8.615817 2.598062705571137 15 6
526


/Users/anmol/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.20.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/anmol/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 0.20.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/anmol/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.20.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/anmol/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 0.20.3 when using versio